# Workflow of Semantic Structure from Motion
## overview
1. Organize data
2. Estimate memory usage
3. Create 2D segmentation using SAM
4. Create projection associations
5. Run object registration

## 1. Organize data
Create a folder under `semantic_SfM/data` and organize your data following the structures below

WebODM:
```
semantic_SfM/data
    ├── box_canyon_park
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── camera.json
        │       ├── shots.geojson
        │       └── model.las   
        ├── segmentations
        └── associations

```

Agisoft:
```
semantic_SfM/data
    ├── box_canyon_park
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── camera_para.xml
        │       └── model.las   
        ├── segmentations
        └── associations
```

## 2. Estimate memory usage

In [ ]:
from ssfm.memory_calculator import memory_calculator

In [ ]:
# pointcloud file
las_file = "../../data/box_canyon_park/SfM_products/model.las"
# image file sample
image_file = "../../data/box_canyon_park/DJI_photos/DJI_0183.JPG"
# number of images
num_images = 129
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

## 3. Create 2D Segmentation using SAM

In [ ]:
from ssfm.image_segmentation import ImageSegmentation
import os

In [ ]:
sam_params = {}
sam_params['model_name'] = 'sam'
sam_params['model_path'] = '../sam/sam_vit_h_4b8939.pth'
sam_params['model_type'] = 'vit_h'
sam_params['device'] = 'cuda:1'
sam_params['points_per_side'] = 64
sam_params['pred_iou_thresh'] = 0.96
sam_params['stability_score_thresh'] = 0.92

In [ ]:
image_segmentor = ImageSegmentation(sam_params)   
image_folder_path = '../../data/box_canyon_park/DJI_photos'
segmentation_folder_path = '../../data/box_canyon_park/segmentations'
image_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
image_segmentor.batch_predict(image_paths, segmentation_folder_path, maximum_size=1000, save_overlap=True)

## 4. Create projection associations

In [1]:
from ssfm.probabilistic_projection import *
import time

In [2]:
pointcloud_projector = PointcloudProjection()
pointcloud_projector.read_pointcloud('../../data/box_canyon_park/SfM_products/model.las')
pointcloud_projector.read_camera_parameters('../../data/box_canyon_park/SfM_products/camera.json', '../../data/box_canyon_park/SfM_products/shots.geojson')

# batch project
image_folder_path = '../../data/box_canyon_park/DJI_photos'
save_folder_path = '../../data/box_canyon_park/associations'

image_list = [f for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
pointcloud_projector.parallel_batch_project(image_list, save_folder_path)

Processing: DJI_0333.JPG
Processing: DJI_0348.JPG
Time for projection:  5.963966608047485
Processing: DJI_0334.JPG
Task 1/129 completed
Time for projection:  6.231230020523071
Processing: DJI_0346.JPG
Task 2/129 completed
Time for projection:  5.64646315574646
Processing: DJI_0262.JPG
Task 3/129 completed
Processing: DJI_0265.JPG
Time for projection:  6.631881475448608
Time for projection:  6.316293954849243
Processing: DJI_0341.JPG
Task 4/129 completed
Task 5/129 completed
Time for projection:  6.696096897125244
Processing: DJI_0308.JPG
Time for projection: Task 6/129 completed
 5.263388395309448
Processing: DJI_0250.JPG
Task 7/129 completed
Time for projection:  6.7626564502716064
Processing: DJI_0281.JPG
Time for projection:  5.084529161453247
Task 8/129 completed
Processing: DJI_0257.JPG
Task 9/129 completed
Time for projection:  5.11978816986084
Processing: DJI_0286.JPG
Task 10/129 completed
Time for projection:  6.008879661560059
Processing: DJI_0259.JPG
Time for projection:  4.4

In [3]:
# build keyimage associations
t1 = time.time()
pointcloud_projector.build_associations_keyimage('../../data/box_canyon_park/associations', '../../data/box_canyon_park/segmentations')
t2 = time.time()
print('Time for building keyimage associations: ', t2 - t1)

Time for building keyimage associations:  102.73334264755249


## 5. Run object registration

In [4]:
from ssfm.object_registration import *
import logging

In [5]:
# Set paths
pointcloud_path = '../../data/box_canyon_park/SfM_products/model.las'
segmentation_folder_path = '../../data/box_canyon_park/segmentations'
image_folder_path = '../../data/box_canyon_park/DJI_photos'
association_folder_path = '../../data/box_canyon_park/associations'


# Create object registration
t1 = time.time()
obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path)
t2 = time.time()
print('Time elapsed for creating object registration: {}'.format(t2-t1))

# Run object registration
obr.object_registration(iou_threshold=0.5)

# save semantic point cloud
obr.save_semantic_pointcloud('../../data/box_canyon_park/semantic_model.las')


Time elapsed for creating object registration: 2.9036059379577637
Processing image 1/129: DJI_0183.npy
Processing image 2/129: DJI_0184.npy
Processing image 3/129: DJI_0241.npy
Processing image 4/129: DJI_0242.npy
Processing image 5/129: DJI_0243.npy
Processing image 6/129: DJI_0244.npy
Processing image 7/129: DJI_0245.npy
Processing image 8/129: DJI_0246.npy
Processing image 9/129: DJI_0247.npy
Processing image 10/129: DJI_0248.npy
Processing image 11/129: DJI_0249.npy
Processing image 12/129: DJI_0250.npy
Processing image 13/129: DJI_0251.npy
Processing image 14/129: DJI_0252.npy
Processing image 15/129: DJI_0253.npy
Processing image 16/129: DJI_0254.npy
Processing image 17/129: DJI_0255.npy
Processing image 18/129: DJI_0256.npy
Processing image 19/129: DJI_0257.npy
Processing image 20/129: DJI_0258.npy
Processing image 21/129: DJI_0259.npy
Processing image 22/129: DJI_0260.npy
Processing image 23/129: DJI_0261.npy
Processing image 24/129: DJI_0262.npy
Processing image 25/129: DJI_02